In [1]:
from neo4j import GraphDatabase, basic_auth
import os

driver = GraphDatabase.driver("bolt://localhost:7687",auth=basic_auth("neo4j", "Kaffe1234"))

In [2]:
def get_count_of_type(label, session):
    q = "MATCH (n:%s) RETURN count(n)" % label
    res = session.run(q)
    print("Added", res.value()[0], "nodes of type", label)
    
def get_count_of_relationship(label, session):
    q = "MATCH ()-[r:%s]-() RETURN count(*)" %label
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", label)

def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("Data/csvs/basal_ganglia/subjects")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

In [3]:
# Deleting all subject data in database
with driver.session() as session:
        session.run('''
            MATCH (n)
            WHERE  n:Specimen
                OR n:Specie 
                OR n:Strain 
                OR n:Substrain 
                OR n:Sex
                OR n:AgeCategory
                OR n:TransgenicLine
            DETACH DELETE n
        ''')
        session.run('''
            DROP CONSTRAINT ON (n:Specimen) ASSERT n.id IS UNIQUE
            DROP CONSTRAINT ON (n:Specie) ASSERT n.id IS UNIQUE
            DROP CONSTRAINT ON (n:Strain) ASSERT n.id IS UNIQUE
            DROP CONSTRAINT ON (n:Substrains) ASSERT n.id IS UNIQUE
            DROP INDEX ON :Specimen(name)
            DROP INDEX ON :Specie(name)
            DROP INDEX ON :Strain(name)
            DROP INDEX ON :Substrains(name)
        ''')
        print("wipeddatabase")

wipeddatabase


In [4]:
# Adding Specimen to graph from specimens.csv

csv_file_path = "file:///%s" % get_csv_path("specimens.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Specimen {id: row.ID, name: row.Specimen_name, form: row.Specimen_form, order: row.Specimen_order})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE INDEX ON :Specimen(name)")
    session.run("CREATE CONSTRAINT ON (n:Specimen) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("Specimen", session)

Added 1065 nodes of type Specimen


In [5]:
# Adding Strain to graph from strains.csv

csv_file_path = "file:///%s" % get_csv_path("strains.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Strain {id: row.ID, name: row.Strain_name, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE INDEX ON :Strain(name)")
    session.run("CREATE CONSTRAINT ON (n:Strain) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("Strain", session)

Added 24 nodes of type Strain


In [6]:
# Adding Substrain to graph from substrains.csv

csv_file_path = "file:///%s" % get_csv_path("substrains.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Substrain {id: row.ID, name: row.Substrain_name, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE INDEX ON :Substrain(name)")
    session.run("CREATE CONSTRAINT ON (n:Substrain) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("Substrain", session)

Added 44 nodes of type Substrain


In [7]:
# Adding Specie to graph from species.csv

csv_file_path = "file:///%s" % get_csv_path("species.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Specie {id: row.ID, name: row.Species, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE INDEX ON :Specie(name)")
    session.run("CREATE CONSTRAINT ON (n:Specie) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("Specie", session)

Added 2 nodes of type Specie


In [8]:
# Adding AgeCategory to graph from age_categories.csv

csv_file_path = "file:///%s" % get_csv_path("age_categories.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:AgeCategory {id: row.ID, name: row.Age_category, ontology: row.Ontological_identifier, description: row.Description})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("AgeCategory", session)

Added 10 nodes of type AgeCategory


In [9]:
# Adding Sex to graph from sex.csv

csv_file_path = "file:///%s" % get_csv_path("sex.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Sex {id: row.ID, name: row.Sex, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Sex", session)

Added 3 nodes of type Sex


In [10]:
# Adding TransgenicLine to graph from transgenic_lines.csv

csv_file_path = "file:///%s" % get_csv_path("transgenic_lines.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:TransgenicLine {id: row.ID, name: row.Transgenic_line_name, RRID: row.Transgenic_line_RRID})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("TransgenicLine", session)

Added 20 nodes of type TransgenicLine


In [11]:
# Relationship SPECIE between Specimen and Specie
csv_file_path = "file:///%s" % get_csv_path("derived_ddr_specimen.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.SpecimenId})
        MATCH (c:Specie { id: row.Species })
        MERGE (a)-[:SPECIE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SPECIE", session)

Added 1006 relationships of type SPECIE


In [12]:
# Relationship STRAIN between Specimen and Strain
csv_file_path = "file:///%s" % get_csv_path("derived_ddr_specimen.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.SpecimenId})
        MATCH (c:Strain { id: row.Strain })
        MERGE (a)-[:STRAIN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("STRAIN", session)

Added 888 relationships of type STRAIN


In [13]:
# Relationship SUBSTRAIN between Specimen and Specie
csv_file_path = "file:///%s" % get_csv_path("derived_ddr_specimen.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.SpecimenId})
        MATCH (c:Substrain { id: row.Substrain })
        MERGE (a)-[:SUBSTRAIN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SUBSTRAIN", session)

Added 240 relationships of type SUBSTRAIN


In [14]:
# Relationship TRANSGENIC_LINE between Specimen and TransgenicLine
csv_file_path = "file:///%s" % get_csv_path("derived_ddr_specimen.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.SpecimenId})
        MATCH (c:TransgenicLine { id: row.Transgenic_line_name })
        MERGE (a)-[:TRANSGENIC_LINE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("TRANSGENIC_LINE", session)

Added 78 relationships of type TRANSGENIC_LINE


In [15]:
# Relationship SEX between Specimen and Sex
csv_file_path = "file:///%s" % get_csv_path("derived_ddr_specimen.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.SpecimenId})
        MATCH (c:Sex { id: row.Sex })
        MERGE (a)-[:SEX]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SEX", session)

Added 796 relationships of type SEX


In [16]:
# Relationship AGE_CATEGORY between Specimen and AgeCategory
csv_file_path = "file:///%s" % get_csv_path("derived_ddr_specimen.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.SpecimenId})
        MATCH (c:AgeCategory { id: row.Sex })
        MERGE (a)-[:AGE_CATEGORY]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("AGE_CATEGORY", session)

Added 796 relationships of type AGE_CATEGORY


In [17]:
# Relationship FROM_SPECIE between Strain and Specie
csv_file_path = "file:///%s" % get_csv_path("strains.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Strain { id: row.ID})
        MATCH (c:Specie { id: row.Species })
        MERGE (a)-[:FROM_SPECIE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("FROM_SPECIE", session)


Added 48 relationships of type FROM_SPECIE
